In [1]:
%load_ext autoreload
%autoreload 2
# %pylab inline
import pandas
import os
os.sys.path.append('../')

## VoxCeleb Dataset


|                 | dev     | test  |
|-----------------|---------|-------|
| # of speakers   | 1,211   | 40    |
| # of videos     | 21,819  | 677   |
| # of utterances | 139,124 | 6,255 |


In [2]:
cnt = 0
data_dir = "/home/muncok/DL/dataset/voxceleb/"
wav_files = {}
all_spks = []
for _, dirnames, _ in os.walk(data_dir):
    for dirname in dirnames:
        if os.path.isfile(os.path.abspath(dirname)): continue
        spk = dirname
        all_spks.append(spk)
        for filename in os.listdir(os.path.join(data_dir,dirname)):
            uniqID = filename.split('.')[0]
            wav_files[uniqID] = (spk, filename)

In [3]:
df = pandas.DataFrame.from_dict(wav_files, 'index')

In [4]:
# set df's column names
df.columns = ['spk', 'file']
df['set'] = ""

In [5]:
# read set devision lists
train_list = []
test_list = []
val_list = []
        
with open("/home/muncok/DL/dataset/voxceleb/Identification_split.txt", 'r') as f:
    for line in f.readlines():
        set_id, filepath = line.split(' ')
        uniqID = filepath.split('/')[-1].rstrip().rstrip(".wav")
        if set_id == '1':
            df.loc[uniqID].set = 'train'
        elif set_id == '2':
            df.loc[uniqID].set = 'val'
        elif set_id == '3':
            df.loc[uniqID].set = 'test'

In [6]:
print("number of speakers: {}".format(len(all_spks)))

number of speakers: 1251


In [8]:
# utterances for each speaker
df.spk.value_counts()

Rob_Reiner              1002
Bob_Barker               656
Alan_Alda                586
Stephen_Fry              570
Louis_C.K.               489
Peter_Jackson            476
Vince_Gilligan           468
J.J._Abrams              426
Quentin_Tarantino        409
David_Attenborough       401
Meat_Loaf                394
Gloria_Steinem           381
Tom_Hooper               377
Lucie_Arnaz              374
James_Woods              364
Shonda_Rhimes            363
Vidya_Balan              357
Kal_Penn                 356
Kriti_Sanon              347
Dick_Van_Dyke            343
Garry_Marshall           340
Neil_deGrasse_Tyson      336
Ricky_Gervais            335
John_Noble               334
Tom_Ford                 332
Irrfan_Khan              330
Kenny_Rogers             326
Claudia_Black            317
Brett_Davern             316
Don_Rickles              314
                        ... 
Damon_Wayans              47
Michael_Weatherly         47
Thomas_Jane               47
Beth_Grant    

In [9]:
new_col = [all_spks.index(x) for x in df.spk]
df = df.assign(label = new_col )

In [10]:
df.to_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Voxc_Dataframes.pkl')

### SV

In [ ]:
# restrict speakers
chosen_spks = df.spk.value_counts().index[:100].tolist()

sv_test = df[(df.set == 'test') & (df.spk.isin(chosen_spks))]
sv_val = df[(df.set == 'val') & (df.spk.isin(chosen_spks))]
sv_train = df[(df.set == 'train') & (df.spk.isin(chosen_spks))]

import csv
tags = ['train', 'val', 'test']
sets = {'train':sv_train, 'val':sv_val, 'test':sv_test}
for tag in tags:
    samples = []
    with open('sv_voxc_{}_manifest.csv'.format(tag), 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = chosen_spks.index(row.spk)          
            sample = ','.join([file_path, str(label+2)]) # 0,1 for speacial purpose
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)